<figure>
    <img src="./figures/Logo.png" style="width:90%">
    <figcaption></figcaption>
</figure>

## In this part of the tutorial, you will
* use one-at-a-time (OAT) sensitivity analysis
* compare RSA and OAT

---

# Tutorial 5a - Sensitivity analysis - one-at-a-time (OAT)

---

## 1 About one-at-a-time sensitivity analysis

TODO write intro

One-at-a-time (OAT) sensitivity analysis is widely applied because it is very simple. It involves varying a certain parameter by a prescribed value up and down by a certain percentage.

## 2 Using one-at-a-time sensitivity analysis

### 2.1 Preparations

**Import python modules**

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sys
sys.path.append('src/')
import HyMod

# Supress warnings from numba
import logging;
logger = logging.getLogger("numba");
logger.setLevel(logging.ERROR)

from ipywidgets import interact, Dropdown, FloatSlider

**Define functions**

In [2]:
def rmse(obs, sim):
    """
    Calculates root mean squared error
    obs :  The observed time series
    sim :  The simulated time series
    @returns : The RMSE
    """
    return np.sqrt(np.mean((obs - sim)**2))

def mean_streamflow(obs, sim):
    """
    Calculates the difference in mean streamflow between observed and simulated
    obs :  The observed time series
    sim :  The simulated time series
    @returns : The absolute mean error
    """
    return np.abs( obs.mean() - sim.mean() )

def hymod_func(param, obj_fun1, obj_fun2):
    # run HyMod
    runoff_sim, states, fluxes = HyMod.hymod_sim(param, P, pet)
    error1 = obj_fun1(runoff_obs, runoff_sim[-n_days:])
    error2 = obj_fun2(runoff_obs, runoff_sim[-n_days:])
    return error1, error2

**Create and display interactive menus for selecting catchment**

In [3]:
catchment_names = ["Santa Cruz Creek, CA, USA", "Siletz River, OR, USA", "Trout River, BC, Canada"]
catchment_dropdown = Dropdown(
    options=catchment_names,
    value=catchment_names[0],
    description='Catchment:',
    disabled=False)

display(catchment_dropdown)

Dropdown(description='Catchment:', options=('Santa Cruz Creek, CA, USA', 'Siletz River, OR, USA', 'Trout River…

**Load data**

In [4]:
# Read catchment data
catchment_name = catchment_dropdown.value
file_dic = {catchment_names[0]: "camels_11124500", catchment_names[1]: "camels_14305500", catchment_names[2]: "hysets_10BE007"}
df_obs = pd.read_csv(f"data/{file_dic[catchment_name]}.csv")
# Make sure the date is interpreted as a datetime object -> makes temporal operations easier
df_obs.date = pd.to_datetime(df_obs['date'], format='%Y-%m-%d')
# Index frame by date
df_obs.set_index('date', inplace=True)
# Select time frame
start_date = '2002-10-01'
end_date = '2003-09-30'
df_obs = df_obs[start_date:end_date]
n_days = 365 # days in three years: 1095
# Reformat the date for plotting
df_obs["date"] = df_obs.index.map(lambda s: s.strftime('%b-%d-%y'))
# Reindex
df_obs = df_obs.reset_index(drop=True)
# Select snow, precip, PET, streamflow and T
df_obs = df_obs[["snow_depth_water_equivalent_mean", "total_precipitation_sum","potential_evaporation_sum","streamflow", "temperature_2m_mean", "date"]]
# Rename variables
df_obs.columns = ["Snow [mm/day]", "P [mm/day]", "PET [mm/day]", "Q [mm/day]", "T [C]", "Date"]

FileNotFoundError: [Errno 2] No such file or directory: 'data/camels_11124500.csv'

**Prepare input data**

In [ ]:
P = df_obs["P [mm/day]"].to_numpy()
pet = df_obs["PET [mm/day]"].to_numpy()
temp = df_obs["T [C]"].to_numpy()
runoff_obs = df_obs["Q [mm/day]"].to_numpy()

parameter_names = ["Sm", "beta", "alpha", "k"]  # define the parameter names
original_parameter_set = [200, 1, .5, .5]  # create the inital parameter set

### 2.2 Using a very simple one-at-a-time sampling strategy

In [ ]:
perturbation_slider = FloatSlider(
    value=10, min=0, max=100, step=1,
    description='Perturbation (%):',
    disabled=False
)
display(perturbation_slider)

In [ ]:
perturbation = perturbation_slider.value  # percentage change for inital
sets = [[200, 1, .5, .5, .5]]  # create list of lists to save parameter sets

# iterate over each parameter and vary it by prescribed value
# the following could be solved much more efficiently by using numpy and matrix calulations
# it is kept simple for educational purposes only
@interact(Sm = (0, 400, 1), beta = (0, 2, 0.01), alpha = (0, 1, 0.01), k = (0, 1, 0.01))    
def oat_hymod_function(Sm=200, beta=1, alpha=0.5, k=0.5):
    original_parameter_set = [Sm, beta, alpha, k]
    output = []
    for i, parameter in enumerate(original_parameter_set):
        parameter_increased =  parameter + parameter * (perturbation/100)  # create an increased version of the current parameter
        parameter_decreased =  parameter - parameter * (perturbation/100)  # create a decreased version of the current parameter
        
        for parameter_changed in [parameter_increased, parameter_decreased]:
            current_parameter_set = original_parameter_set.copy()  # copy the original set to define the current parameter set (which will be changed)
            current_parameter_set[i] = parameter_changed  # enter the in-/decreased parameter into the parameter set
            Rf = 1 - current_parameter_set[-1]  # define Rf (= 1-Rs); Rs is at the last position of the parameter set, that is [-1] in python
            current_parameter_set.append(Rf)  # add Rf to the parameter set
            sets.append(current_parameter_set.copy())  # add the parameter set to the list of sets
    
            actual_perturbation = 100 * ((parameter_changed / parameter) - 1)  # compute the actual perturbation (positive or negative)
            # Run HyMod, and get the ouput
            rmse_v, msf_v = hymod_func(current_parameter_set, rmse, mean_streamflow)
            output.append({"Objective function result": rmse_v, "Objective": "RMSE", 
                           "Parameter": parameter_names[i], "Perturbation (%)": actual_perturbation})
            output.append({"Objective function result": msf_v, "Objective": "Bias", 
                           "Parameter": parameter_names[i], "Perturbation (%)": actual_perturbation})
        
    df = pd.DataFrame.from_records(output)
    
    # plot the data
    rel = sns.relplot(data=df, x="Perturbation (%)", y="Objective function result", col="Parameter", hue="Objective", height=3, aspect=0.7)
    #rel.set(ylim=(0, 15))
    plt.show()

#for set in zip(sets):  # uncomment to see the sets of parameters
#    print(set)

#### Exercise 1

1.1 A simple OAT application is implemented in the above. It perturbates the four parameters of the HyMOD model individually by a percentage you can choose with the slider. What does the plot tell you? 

* Answer

1.2 What if we chose a different starting point of parameter values for our analysis? You can change the original input parameter set using the sliders.

* Answer

1.3 Can you say which parameter is most sensitive? 

* Answer


### 2.3 Using another simple one-at-a-time sampling strategy

In [ ]:
step_slider = FloatSlider(
    value=100, min=5, max=100, step=5,
    description='Step size:',
    disabled=False
)
display(step_slider)

In [ ]:
perturbations = np.arange(10, 100, step_slider.value)  # percentage changes to alter inital parameters by
sets = [[200, 1, .5, .5, .5]] # create list of lists to save parameter sets

# A very simple One-at-a-time sampling strategy
@interact(Sm = (0, 400, 1), beta = (0, 2, 0.01), alpha = (0, 1, 0.01), k = (0.01, 0.5, 0.01))    
def oat_hymod_function(Sm=200, beta=1, alpha=0.5, k=0.25):
    original_parameter_set = [Sm, beta, alpha, k]
    output = []
    for i, parameter in enumerate(original_parameter_set):
        # iterate over each parameter
        for perturbation in perturbations:
            # and vary it by prescribed value
            # the following could be solved much more efficiently by using numpy and matrix calulations
            # it is kept simple for educational purposes only
            parameter_increased =  parameter + parameter * (perturbation/100)  # create an increased version of the current parameter
            parameter_decreased =  parameter - parameter * (perturbation/100)  # create a decreased version of the current parameter
        
            for parameter_changed in [parameter_increased, parameter_decreased]:
                current_parameter_set = original_parameter_set.copy()  # copy the original set to define the current parameter set (which will be changed)
                current_parameter_set[i] = parameter_changed  # enter the in-/decreased parameter into the parameter set
                Rf = 1 - current_parameter_set[-1]  # define Rf (= 1-Rs); Rs is at the last position of the parameter set, that is [-1] in python
                current_parameter_set.append(Rf)  # add Rf to the parameter set
                sets.append(current_parameter_set.copy())  # add the parameter set to the list of sets
                
                actual_perturbation = 100 * ((parameter_changed / parameter) - 1)  # compute the actual perturbation (positive or negative)
                
                rmse_v, msf_v = hymod_func(current_parameter_set, rmse, mean_streamflow)  # run HyMod, and get the ouput
                output.append({"Objective function result": rmse_v, "Objective": "RMSE", 
                               "Parameter": parameter_names[i], "Perturbation (%)": actual_perturbation})
                output.append({"Objective function result": msf_v, "Objective": "Bias", 
                               "Parameter": parameter_names[i], "Perturbation (%)": actual_perturbation})
            
    df = pd.DataFrame.from_records(output)
    # plot the data
    rel = sns.relplot(data=df, x="Perturbation (%)", y="Objective function result", col="Parameter", hue="Objective", height=3)
    #rel.set(ylim=(0, 15))
    plt.show()
    
    #for set in zip(sets):  # uncomment to see the sets of parameters
    #    print(set)

#### Exercise 2

2.1 Use the step slider to change the step-size and test for more variations! Are the results easier to interpret? 

* Answer

2.2 What if we chose a different starting point of parameter values for our analysis? You can change the original input parameter set using the sliders.

* Answer

2.3 Do different objective functions react in the same way?

* Answer

#### Exercise 3

3.1 How do the results differ from RSA?

* Answer

3.2 Think back to the lecture and the four purposes of sensitivity analysis (Screening, Ranking, Mapping, Variance Cutting), for which one of the four is RSA and OAT useful/not useful?

* Answer

#### Additional task

Add different objective functions and analyse the results
